#1: Import Libraries

In [54]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

np.random.seed(42)


#2: Read, explore and process data

In [ ]:
data = pd.read_csv("Live.csv")
X = data[data['status_type'] == 'video']
df = X.drop(['status_id', 'status_type', 'status_published', 'Column1', 'Column2', 'Column3', 'Column4'], axis=1)

#3: Explore data

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2334 entries, 0 to 7034
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   num_reactions  2334 non-null   int64
 1   num_comments   2334 non-null   int64
 2   num_shares     2334 non-null   int64
 3   num_likes      2334 non-null   int64
 4   num_loves      2334 non-null   int64
 5   num_wows       2334 non-null   int64
 6   num_hahas      2334 non-null   int64
 7   num_sads       2334 non-null   int64
 8   num_angrys     2334 non-null   int64
dtypes: int64(9)
memory usage: 182.3 KB


In [57]:
df.isnull().sum()

num_reactions    0
num_comments     0
num_shares       0
num_likes        0
num_loves        0
num_wows         0
num_hahas        0
num_sads         0
num_angrys       0
dtype: int64

In [58]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)


#4: Code K-Means

In [ ]:
def Random_choices(X, K):
    row_indices = np.random.choice(X.shape[0], K, replace=False) 
    return X[row_indices]  




def Distance(X, K, centroids):
    distances = np.zeros((X.shape[0], K))
    for i in range(0, K):
        distances[:, i] = np.sum((X - centroids[i]) ** 2, axis=1) 
    return distances 


def fit(X, K):
    centroid = Random_choices(X, K)
    for _ in range(0, 30):
        distances = Distance(X, K, centroid)
        labels = np.argmin(distances, axis=1) 
        m = X.shape[0]
        new_centroids = np.array([X[labels == k, :].mean(axis=0) for k in range(K)])
        if np.allclose(centroid, new_centroids):
            break
        centroid = new_centroids
    return labels

    
    

#5: Classification

In [60]:
max_score = 0
final_K = 3
for K in range(3, 20):
    cur_label = fit(df_scaled, K)    
    kmeans = KMeans(n_clusters=K, init='k-means++', n_init=30, random_state=42)
    cur_label_tool = kmeans.fit_predict(df)
    ari_score = adjusted_rand_score(cur_label, cur_label_tool)
    if ari_score > max_score:
        max_score = ari_score
        final_K = K


final_label = fit(df_scaled, final_K)    
kmeans = KMeans(n_clusters=final_K, init='k-means++', n_init=30, random_state=42)
final_label_tool = kmeans.fit_predict(df)

print("Final K:", final_K)
print("Custom K-Means Labels:", final_label)
print("Sklearn K-Means Labels:", final_label_tool)

ari_score = adjusted_rand_score(final_label, final_label_tool)
print("Adjusted Rand Index (ARI):", ari_score)  

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12876\4170593404.py:22: RuntimeWarning: Mean of empty slice.
  new_centroids = np.array([X[labels == k, :].mean(axis=0) for k in range(K)])
c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:137: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12876\4170593404.py:22: RuntimeWarning: Mean of empty slice.
  new_centroids = np.array([X[labels == k, :].mean(axis=0) for k in range(K)])
c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:137: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


Final K: 12
Custom K-Means Labels: [9 7 2 ... 4 7 7]
Sklearn K-Means Labels: [11 11 11 ... 10  1  1]
Adjusted Rand Index (ARI): 0.7358005602254624


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12876\4170593404.py:22: RuntimeWarning: Mean of empty slice.
  new_centroids = np.array([X[labels == k, :].mean(axis=0) for k in range(K)])
c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:137: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
